In [2]:
import numpy as np
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd

# P1 data
data1_path = '../Datasets/TaxiBJ/TAXIBJ2013.grid'
# P2 data
data2_path = '../Datasets/TaxiBJ/TAXIBJ2014.grid'
# P3 data
data3_path = '../Datasets/TaxiBJ/TAXIBJ2015.grid'
# P4 data
data4_path = '../Datasets/TaxiBJ/TAXIBJ2016.grid'
# external data
extdatapath = '../Datasets/TaxiBJ/TAXIBJ.ext'

# load P1 data
xdata1 = pd.read_csv(data1_path)
xdata1.head()
# load P2 data
xdata2 = pd.read_csv(data2_path)
# load P3 data
xdata3 = pd.read_csv(data3_path)
# load P4 data
xdata4 = pd.read_csv(data4_path)
# load external data
extdata = pd.read_csv(extdatapath)
extdata.shape
extdata = extdata.drop_duplicates(subset=['time'], keep='first')
extdata.shape

(58977, 21)

In [3]:
#xdata1 = xdata1.drop(columns=['dyna_id', 'type'])
xdata1_gb = xdata1.groupby('time')#.apply(print)
xdata2_gb = xdata2.groupby('time')
xdata3_gb = xdata3.groupby('time')#.apply(print)
xdata4_gb = xdata4.groupby('time')
# number of groups
xdata1_gb.ngroups
xdata2_gb.ngroups
xdata3_gb.ngroups
xdata4_gb.ngroups

# convert to list
xdata1_gb = list(xdata1_gb)
xdata2_gb = list(xdata2_gb)
xdata3_gb = list(xdata3_gb)
xdata4_gb = list(xdata4_gb)


4848

4368

5520

6624

In [179]:
# convert data to numpy array and save as file
for j in range(1,5):
    size = len(eval(f'xdata{j}_gb'))
    flow_data = np.empty([size, 2, 32, 32])
    ext_data = np.empty([size, 27]) # 7 day of week, holiday, temp, windspeed, 17 weathers
    for i in range(size):
        # flow data
        data = eval(f'xdata{j}_gb[i][1]')
        flowcont.append(data['time'])
        data_inflow = data.pivot_table(values='inflow', index='row_id', columns='column_id').to_numpy()
        data_outflow = data.pivot_table(values='outflow', index='row_id', columns='column_id').to_numpy()
        flow_data[i, 0] = data_inflow
        flow_data[i, 1] = data_outflow
        # external data
        time = eval(f'xdata{j}_gb[i][0]')
        date = pd.to_datetime(time, format='%Y-%m-%dT%H:%M:%SZ').dayofweek
        days = np.zeros([1, 8])
        days[0, date] = 1
        if date > 4: # is a weekend date
            days[0, 7] = 1 # holiday index
        edata = extdata.loc[extdata['time']==time].drop(columns=['ext_id', 'time'])
        ext_data[i] = np.concatenate((days, edata), axis=1)
    np.save(f'../Datasets/TaxiBJ{2012+j}', flow_data)
    np.save(f'../Datasets/TaxiBJext{2012+j}', ext_data)


In [180]:
# split TaxiBJ2016 in half since file exceeds 100MB
data = np.load('../Datasets/TaxiBJ2016.npy')
half = int(len(data)/2)
data1 = data[0:half]
data2 = data[half:]
np.save(f'../Datasets/TaxiBJ2016_1', data1)
np.save(f'../Datasets/TaxiBJ2016_2', data2)

In [29]:
# verify sizes
data = np.load('../Datasets/TaxiBJ2013.npy')
data2 = np.load('../Datasets/TaxiBJext2013.npy')
data.shape
data2.shape

(4848, 2, 32, 32)

(4848, 27)

In [28]:
# output days for which there are data for
for j in range(1, 5):
    xdatatime = []
    for i in range(0, len(eval(f'xdata{j}_gb')), 48):
        xdatatime.append(eval(f'xdata{j}_gb')[i][0][:-10])
    with open(f'./Datasets/TaxiBJ/days{2012+j}.txt', 'w') as fp:
        for item in xdatatime:
            day = pd.to_datetime(item, format='%Y-%m-%d').day_name()
            fp.write(item + ',' + day + '\n')

18

19

21

20

18

...

19

21

20

18

20